In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta
import cvxpy as cp


## Group Assignment
### Team Number: 07
### Team Member Names: Wendi Xue, Ammar Adam, Maximilian Bartoszek
### Team Strategy Chosen: Risk-Free(Market Beat, Market Meet, Risk-Free)

### Loading and filtering valid tickers
In the secret list of tickers that will be used to generate our portfolio, we will take into consideration the following factors:
- If there are any duplicate tickers in the list, we will drop them.
- If there are any tickers in the list that do not exist, using the function info.get() will prevent any KeyErrors
- We only want valid US and Canadian companies that have listed stocks in the markets, so we will only take stocks with United States or Canada as their country and CAD and USD as their currency

In [10]:
# Load the tickers
df = pd.read_csv('Tickers_Example.csv', header=None) # test with Tickers_Example.csv?
# If there are any duplicate tickers, we will drop them
df = df.drop_duplicates()
ticker_lst = list(df.iloc[:,0])

# Filter out tickers that don't exist; include only valid tickers
valid_tickers_lst = []
for ticker in ticker_lst:
    try: 
        ticker_data = yf.Ticker(ticker) # call up data
        country = ticker_data.info.get('country') 
        currency = ticker_data.info.get('currency')
        if country in {'Canada','United States'} and currency in {'CAD','USD'}: # filters us and canadian tickers and listed stocks
            valid_tickers_lst.append(ticker)
    except Exception as err:
        print(f"Error: {err}")

print(valid_tickers_lst)

['AAPL', 'ABBV', 'ABT', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'PRL.TO']


## Average daily volume constraint
- If the stock does not have any information on their volume on a specific day, we will drop that day.
- Between October 1, 2024 and September 30, 2025, we will group together the data of the volume by months to see if there are months with less than 18 trading days. If there is we will drop that month when calculating the average daily volume.
- If any stock has an average daily volume below 5,000 shares we will exclude that stock from the list of tickers.

In [11]:
# Apply a filter to remove all stocks with volumes below 5000 (in the given period)
# period in which we look at the volumes 
volume_start_date = '2024-10-01'
volume_end_date = '2025-09-30' 
filtered_lst = []

for ticker in valid_tickers_lst: # goes through every ticker to filter
    data = yf.download(
        tickers=ticker,
        start=volume_start_date,
        end=volume_end_date,
        auto_adjust=True
    )
    volume_data = data[['Volume']].dropna() # volume data

    keep_months = pd.DataFrame() # df of all the months with more than 18 trading days
    volume_data['Month'] = volume_data.index.to_period('M') # create new column of index only by (YYYY-MM)
    grouped_month_index = volume_data.groupby(['Month']) # group data by month

    for month, group in grouped_month_index:
        if len(group) >= 18: # if the month has more than 18 trading days
            keep_months = pd.concat([keep_months, group]) # add data of months with more than 18 trading days

    average_daily_volume = keep_months['Volume'][ticker].mean() # calculate average daily volume
    if average_daily_volume >= 5000: # determine if above or below 5000 shares
        filtered_lst.append(ticker) # add to filtered list if greater or equal to 5000 shares

print(filtered_lst)

# Get the daily data over chosen timeframe (2025-10-24 to 2025-10-31 for testing?)
start_date = '2025-10-24' # change to Nov 21 2025
end_date = '2025-10-31' # change to Nov 28 2025

daily_data = yf.download(
    tickers=filtered_lst,
    start=start_date,
    end=end_date, 
    auto_adjust=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

['AAPL', 'ABBV', 'ABT', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'PRL.TO']


[*********************100%***********************]  37 of 37 completed


In [12]:
# Get/calculate volatility (std), beta, market cap, and sectors

metrics_df = pd.DataFrame(columns=['Ticker', 'Volatility', 'Beta', 'MarketCap', 'Sector'])

# ---- Market index for beta ----
market_index = "^GSPC"
market_hist = yf.download(market_index, start=start_date, end=end_date)
market_prices = market_hist["Close"].dropna()
market_returns = market_prices.pct_change().dropna()

# ---- Loop through filtered tickers ----
for ticker in filtered_lst:

    try:
        prices = daily_data['Adj Close'][ticker]
    except KeyError:
        prices = daily_data['Close'][ticker]

    prices = prices.dropna()
    if prices.empty:
        continue

    returns = prices.pct_change().dropna()
    if returns.empty:
        continue

    volatility = returns.std(ddof=0)     # daily std

    aligned = pd.concat([returns, market_returns], axis=1, join="inner")
    aligned.columns = ["Stock", "Market"]

    # covariance(stock, market)
    cov_sm = aligned.cov(ddof=0).iloc[0,1]

    # variance(market)
    var_market = aligned["Market"].var(ddof=0)

    if var_market == 0 or pd.isna(cov_sm):
        beta = np.nan
    else:
        beta = cov_sm / var_market

    info = yf.Ticker(ticker).info
    mcap = info.get("marketCap", np.nan)
    sector = info.get("sector", "Unknown")

    metrics_df.loc[len(metrics_df)] = [
        ticker,
        volatility,
        beta,
        mcap,
        sector
    ]

metrics_df = metrics_df.dropna(subset=['Volatility', 'MarketCap']).reset_index(drop=True)

print(metrics_df)

# Use the weighted scoring algorithm in the doc to provide a score /100 per stock; take from google docs
# Ammar

scored_df = metrics_df.copy()

# Volatility scoring function
def vol_points(v):
    # v is daily volatility (e.g., 0.02 = 2%)
    if v < 0.02:
        return 45
    elif v < 0.03:
        return 40
    elif v < 0.04:
        return 35
    elif v < 0.05:
        return 25
    elif v < 0.06:
        return 15
    else:
        return 5

# Beta scoring function
def beta_points(b):
    if pd.isna(b):
        return 20  # neutral if missing
    if b < 0.6:
        return 35
    elif b < 0.9:
        return 25
    elif b < 1.1:
        return 20
    elif b < 1.3:
        return 10
    else:
        return 0

# Market cap scoring function
def cap_points(m):
    # thresholds in dollars
    if m > 200e9:
        return 20
    elif m >= 50e9:
        return 16
    elif m >= 10e9:
        return 12
    elif m >= 2e9:
        return 8
    else:
        return 4

# Apply scoring table
scored_df['VolPts']  = scored_df['Volatility'].apply(vol_points)
scored_df['BetaPts'] = scored_df['Beta'].apply(beta_points)
scored_df['CapPts']  = scored_df['MarketCap'].apply(cap_points)

# Final score out of 100
scored_df['Score'] = scored_df['VolPts'] + scored_df['BetaPts'] + scored_df['CapPts']

print(scored_df[['Ticker','Volatility','Beta','MarketCap','Sector','Score']])

# After scoring, put all stocks in lists based on sector
# Ammar

sector_dict = {}   # dictionary: sector → list of tickers

for _, row in scored_df.iterrows():
    sector = row['Sector']
    ticker = row['Ticker']
    
    # create key if doesn't exist
    if sector not in sector_dict:
        sector_dict[sector] = []
    
    # append ticker
    sector_dict[sector].append(ticker)

# Preview the grouping
for sec, tics in sector_dict.items():
    print(f"{sec}: {tics}")

C:\Users\maxba\AppData\Local\Temp\ipykernel_9500\2207700544.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_hist = yf.download(market_index, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


     Ticker  Volatility      Beta      MarketCap                  Sector
0      AAPL    0.008717  0.729520  4022784491520              Technology
1      ABBV    0.008665 -0.554801   404342243328              Healthcare
2       ABT    0.007958  0.093310   216269438976              Healthcare
3       AIG    0.006295  0.028356    42217865216      Financial Services
4      AMZN    0.018096  2.008883  2381780090880       Consumer Cyclical
5       AXP    0.006994  0.461516   241290199040      Financial Services
6        BA    0.029839  3.358293   138816241664             Industrials
7       BAC    0.006427  0.016106   386605613056      Financial Services
8     BB.TO    0.006029  0.600508     3525568000              Technology
9      BIIB    0.008653 -0.362861    24964325376              Healthcare
10       BK    0.008580 -0.212121    75926233088      Financial Services
11      BLK    0.009149 -0.121249   156950659072      Financial Services
12      BMY    0.034901 -3.632762    92769263616   

## Forming the top 25 stocks dataframe
- To maximize our diversification, we will select the top 25 stocks according to the scores allocated by our scoring algorithm, which is the maximum amount we are allowed to have. This will protect our portfolio from single-company risk.
- Using the concept of diversification, we will also spread our selection of stocks accross all sectors so that our portfolio is not entirely dependent on the how well a specific sector performs. Thus, we will pick the top 5 stocks, or the maximum amount of stocks if there are less than 5, with the best score from each sector and from that, we will pick the top 25 stocks with the best scores. This way we will not have more than 5 stocks per sector.


In [13]:
# Take the top 5 from each sector (based on their score /100) and put them in a new dataframe
def sector_top5(df,sector):
    top5_count = 0
    sector_top5 = pd.DataFrame() # top 5 tickers of a sector with the best scoring
    for i in range(len(df)):
        if df.iloc[i]['Sector'] == sector:
            top5_count += 1
            if top5_count > 5:
                top5_count = 5
                smallest_score = sector_top5['Score'].min()
                smallest_score_index = sector_top5['Score'].idxmin()
                if df.iloc[i]['Score'] > smallest_score:
                    sector_top5.loc[smallest_score_index] = df.iloc[i]
            else:
                sector_top5 = pd.concat([sector_top5,df.iloc[[i]]])
    return sector_top5

sector_lst = list(sector_dict.keys()) # creates a list of all the sectors
all_sectors_top5 = pd.DataFrame()
for sector in sector_lst:
    df_sector_top5 = sector_top5(scored_df,sector)
    all_sectors_top5 = pd.concat([all_sectors_top5,df_sector_top5])

# Then return the top 25
# sort stocks from highest score to lowest
all_sectors_top5 = all_sectors_top5.sort_values('Score', ascending=False)
top_25_stocks = all_sectors_top5.head(25) # top 25 stocks
# reset index 
top_25_stocks = top_25_stocks.reset_index(drop=True)
print(top_25_stocks)

     Ticker  Volatility      Beta      MarketCap                  Sector  \
0       LLY    0.018934 -1.635101   937402499072              Healthcare   
1      ABBV    0.008665 -0.554801   404342243328              Healthcare   
2       ABT    0.007958  0.093310   216269438976              Healthcare   
3       MRK    0.006050  0.389409   235478532096              Healthcare   
4        PM    0.007728 -0.947957   242215796736      Consumer Defensive   
5       BAC    0.006427  0.016106   386605613056      Financial Services   
6        PG    0.008120 -0.389964   345882787840      Consumer Defensive   
7        KO    0.013712 -0.054101   305797070848      Consumer Defensive   
8       AXP    0.006994  0.461516   241290199040      Financial Services   
9     TD.TO    0.008077  0.214897   198739509248      Financial Services   
10      PFE    0.005039  0.009906   139271421952              Healthcare   
11      PEP    0.015298 -0.051315   199926349824      Consumer Defensive   
12       CL 

## Market Cap Mix: Check for a small-cap
To meet the requirement of at least one small-cap:
- Check if the 25th stock in our dataframe is a small-cap (we will need the last row of the dataframe to be a small-cap for the mininum variance portfolio optimization). If it is we will proceed with the minimum varaicnace portfolio optimization.
- If the last row is not a small-cap, check if there already is a small-cap in the entire top 25 stocks. If there is a small-cap, we will move the first small-cap we find in the dataframe to the last row to fufill the requirement for our minimum variance optimization.
- If our top 25 stocks dataframe does not contain a small-cap, we will take the best scoring small-cap from the entire list of tickers and we will replace the last row of our top 25 stocks with it.

In [14]:
# check for small cap in the last row of the top 25 stocks as code determining weight of portfolio will consider this
small_cap = 2e9
last_row_market_cap = top_25_stocks['MarketCap'].iloc[-1]
market_cap = top_25_stocks['MarketCap']
print(last_row_market_cap)
if last_row_market_cap < small_cap:
    print("Small cap in the 25th row found")
# move small cap to the last row if there is a small cap in the top 25 stocks
elif (market_cap < small_cap).any():
    for row in top_25_stocks.itertuples():
        if row.MarketCap < small_cap:
            top_25_stocks = top_25_stocks.drop(row)
            top_25_stocks = pd.concat([top_25_stocks,row])
# get the best scoring small cap from the dataframe of the top 5 of all sectors and replace it with the stock of the last row of the top 25 stocks            
else:
    small_cap_df = scored_df[scored_df['MarketCap'] < small_cap].copy()   
    small_cap_df = small_cap_df.sort_values('Score', ascending=False)
    top_score_small_cap = small_cap_df.iloc[0]
    # replace the last stock of the top 25 with the small cap with the best score
    top_25_stocks.iloc[-1] = top_score_small_cap
print(top_25_stocks)

138816241664
     Ticker  Volatility      Beta      MarketCap                  Sector  \
0       LLY    0.018934 -1.635101   937402499072              Healthcare   
1      ABBV    0.008665 -0.554801   404342243328              Healthcare   
2       ABT    0.007958  0.093310   216269438976              Healthcare   
3       MRK    0.006050  0.389409   235478532096              Healthcare   
4        PM    0.007728 -0.947957   242215796736      Consumer Defensive   
5       BAC    0.006427  0.016106   386605613056      Financial Services   
6        PG    0.008120 -0.389964   345882787840      Consumer Defensive   
7        KO    0.013712 -0.054101   305797070848      Consumer Defensive   
8       AXP    0.006994  0.461516   241290199040      Financial Services   
9     TD.TO    0.008077  0.214897   198739509248      Financial Services   
10      PFE    0.005039  0.009906   139271421952              Healthcare   
11      PEP    0.015298 -0.051315   199926349824      Consumer Defensive   

# Minimum Variance Portfolio Optimization

Now that we have selected the 25 stocks we would like to invest in, it is time to determine how much to allocate into each stock. This part constructs a minimum-variance portfolio that aims to achieve the lowest possible volatility while following the constraints listed in the assignment. Rather than chasing maximum returns, this strategy prioritizes preserving our wealth and reducing risk. 
 
# Why Minimum Variance?
The main idea is that by combining assets with different return patterns, we can reduce overall portfolio risk below what we would have if we put the same amount in every stock. This happens because stock prices don't all move together. When some stocks fall, others may rise or remain stable. We can use this to our advantage to invest in two assets that react oppositely to market news, that way if one moves, the other will too, just in the other direction. Lower volatility means our portfolio moves less and stays close to a 0% return.

# Why Optimization Instead of Monte Carlo Simulation?
 
You, like my teammates, might wonder, "Why not just run Monte Carlo simulations to try millions of random portfolio combinations?" 
Here's why MVP optimization is superior:

- Optimization finds the mathematically proven minimum variance portfolio every time. Meanwhile Monte Carlo might get close but is never guaranteed to find the true optimum
- MVP Opt solves problems in seconds using convex optimization algorithms. Monte Carlo needs millions of iterations to get the same answer, taking much longer
- Optmization returns the same portfolio every time, while Monte Carlo will give different ones each time based on the given seed

# Our Strategy
We implement a three-stage process:
1. Find the theoretical minimum variance portfolio from our list of stocks
2. Apply the constraints and refine to ensure a proper portfolio
3. Use mvp optimization a second time and return the results 

In [15]:
tickers = top_25_stocks

sectors = {
    # Tech
    "AAPL":"Tech","MSFT":"Tech","AMZN":"Tech","GOOGL":"Tech","META":"Tech",
    "NVDA":"Tech","AMD":"Tech","INTC":"Tech","ADBE":"Tech","CRM":"Tech",
    "CSCO":"Tech","ORCL":"Tech","QCOM":"Tech","NFLX":"Tech","SHOP":"Tech",
    "SNOW":"Tech","PLTR":"Tech","UBER":"Tech","ABNB":"Tech","PANW":"Tech",

    # Healthcare
    "JNJ":"Healthcare","PFE":"Healthcare","UNH":"Healthcare","LLY":"Healthcare","MRK":"Healthcare",
    "BMY":"Healthcare","GILD":"Healthcare","AMGN":"Healthcare","REGN":"Healthcare","VRTX":"Healthcare",
    "ISRG":"Healthcare","ABT":"Healthcare","MDT":"Healthcare","CVS":"Healthcare","MCK":"Healthcare",

    # Financials
    "JPM":"Finance","BAC":"Finance","C":"Finance","GS":"Finance","MS":"Finance","WFC":"Finance",
    "TD":"Finance","RY":"Finance","BNS":"Finance","CM":"Finance","BMO":"Finance","BN":"Finance",
    "AIG":"Finance","TRV":"Finance","ALL":"Finance","CB":"Finance","ICE":"Finance","CME":"Finance",
    "MSCI":"Finance","SPGI":"Finance",

    # Energy
    "XOM":"Energy","CVX":"Energy","COP":"Energy","DVN":"Energy","EOG":"Energy",
    "ENB":"Energy","TRP":"Energy","CNQ":"Energy","SU":"Energy","PSX":"Energy","VLO":"Energy",

    # Industrials
    "CAT":"Industrials","DE":"Industrials","GE":"Industrials","MMM":"Industrials","BA":"Industrials",
    "LMT":"Industrials","NOC":"Industrials","RTX":"Industrials","GD":"Industrials","ETN":"Industrials",
    "CP":"Industrials","CNR":"Industrials","TFII":"Industrials","WM":"Industrials","FDX":"Industrials",

    # Consumer Staples
    "WMT":"Staples","COST":"Staples","TGT":"Staples","KO":"Staples","PEP":"Staples",
    "PG":"Staples","CL":"Staples","MO":"Staples","TAP":"Staples","KR":"Staples",

    # Consumer Discretionary
    "HD":"Discretionary","LOW":"Discretionary","SBUX":"Discretionary","MCD":"Discretionary","NKE":"Discretionary",
    "LULU":"Discretionary","TSLA":"Discretionary","F":"Discretionary","GM":"Discretionary","RCL":"Discretionary",

    # Utilities
    "NEE":"Utilities","DUK":"Utilities","SO":"Utilities","AEP":"Utilities","BEP":"Utilities",
    "AQN":"Utilities","FTS":"Utilities","EMA":"Utilities",

    # Communications
    "T":"Communications","VZ":"Communications","TMUS":"Communications","CHTR":"Communications",
    "CMCSA":"Communications","WBD":"Communications","BCE":"Communications",

    # Materials
    "NEM":"Materials","FCX":"Materials","TECK":"Materials","NTR":"Materials",
    "APD":"Materials","LIN":"Materials","DOW":"Materials",

    # Real Estate
    "PLD":"Real Estate","AMT":"Real Estate","EQIX":"Real Estate","O":"Real Estate","SPG":"Real Estate"
}


# Portfolio Constraints

| Constraint | How | Explanation |
|------------|-------|-----------|
| **No Short Selling** | Weights >= 0 | We only buy stocks (no betting against them) |
| **Max Weight per Stock** | Weights <= 15% | Prevents over-concentration in any single company |
| **Min Weight per Stock** | 1 / (2 * num_tickers) | Ensures we don't hold like 0.001% of a company |
| **Max Sector Exposure** | 40% | Prevents industry-specific risk |
| **Minimum Holdings** | 10-25 stocks | Ensures adequate diversification |
| **Small-Cap Requirement** | >= 1 stock | Adds diversification |


In [16]:
# Values needed for constraints
num_tickers = len(tickers)
min_weight_if_included = 1 / (2 * num_tickers)  
max_weight = 0.15 # No stock can exceed 15%
max_sector_weight = 0.40 # No sector can exceed 40%
min_stocks = 10 # Must have at least 10 stocks
small_epsilon = 0.001 # Threshold to remove negligible stocks

# Data Collection & Processing

We download one year of historical price data to estimate the covariance matrix. One year will provide us with enough data points to understand the typical market behavior while remaining relevant to the current market conditions.

We will try using adjusted prices as they account for stock splits and dividends. Without adjustment, a 2-for-1 stock split would wrongly appear as a 50% loss.

In [17]:
# Download historical data
end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

prices_all = yf.download(tickers, start=start_date, end=end_date, progress=False, auto_adjust=True)

# Tries to get adjusted closes if possible
# Adjusted close is better than close in case of stock splits
if isinstance(prices_all.columns, pd.MultiIndex):
    if 'Adj Close' in prices_all.columns.get_level_values(0):
        prices = prices_all['Adj Close'].copy()
    else:
        prices = prices_all['Close'].copy()
else:
    prices = prices_all.copy()

# Filters out tickers that don't have yf data
available = [t for t in tickers if t in prices.columns]

prices = prices[available]   
tickers = available
num_tickers = len(tickers)

AttributeError: 'DataFrame' object has no attribute 'split'

# Covariance Matrix Estimation
 
The **covariance matrix** is the heart of our portfolio optimization. It helps us to model:
- Variance: How volatile each individual stock is
- Covariance: How different stocks move with one another

What this means:
- If two stocks have a positive covariance they will tend to move in the same direction
- If two stocks have a negative covariance they will move in opposite directions (great for our diversification woohoooo!)
- If they have NO covariance, their stock movements will be completely independent from one another

The minimum variance portfolio optimizer uses this matrix to find combinations of stocks that offset each other's volatility.

In [ ]:
# Calculate returns and covariance matrix
returns = prices.pct_change(fill_method=None).dropna(how='all')
cov_matrix = returns.cov().values

# Map tickers to sectors in order for future constraint reinforcement
ticker_sectors = [sectors[t] for t in tickers]
unique_sectors = sorted(set(ticker_sectors))

# Step 1: Initial Optimization

Minimum variance optimization solves this mathematical problem:
 
# **σ²ₚ = wᵀΣw**

Where:
- wT = the transposed vector of portfolio weights (essentially just how much to invest in each stock)
- Σw = the covariance matrix (the thing we made just now that shows the risk relationships between stocks)
- σ²ₚ = portfolio variance (we want this to be as low as possible)
 
The optimizer searches through all possible weight combinations to find the one that produces the lowest and least volatile returns. It uses negative correlations and low covariances to cancel out individual stock volatility.
 
For our use of it, we have implemented the CVXPY library to solve this.

In [ ]:
# Minimizing Portfolio Variance (THANK YOU CVXPY)
w = cp.Variable(num_tickers)
objective = cp.Minimize(cp.quad_form(w, cov_matrix))

# Basic constraints
constraints = [
    cp.sum(w) == 1, # Weights sum to 100%
    w >= 0, # No short selling
    w <= max_weight # Max 15% per stock
]

# Add Sector Constraints (40% max)
for sector in unique_sectors:
    indices = []
    for i in range(len(tickers)):
        if sectors[tickers[i]] == sector:
            indices.append(i)
    
    if indices:
        total_sector_weight = cp.sum(w[indices])
        constraints.append(total_sector_weight <= max_sector_weight)

# Solve the optimization problem (I LOVE YOU CVXPY)
problem = cp.Problem(objective, constraints)
result = problem.solve()

# Step 2: Cleaning and Portfolio Refinement

In testing, the optimizer often assigned tiny weights to many stocks (Like 0.001% D:). These are impractical so we zero out positions below 0.1% and ensure we have at least 10 holdings in our portfolio. 

In [ ]:
# Zero out companies with really small allocations
raw_weight = np.maximum(w.value, 0)  
raw_weight[raw_weight < small_epsilon] = 0

# Makes list of all tickers that actually have value
included_idx = np.where(raw_weight > 0)[0].tolist()

# Ensure we have at least 10 stocks
# If not, pick top weights until we have the minimum number of stocks
if len(included_idx) < min_stocks:
    biggest = []
    for i in range(len(raw_weight)):
        biggest.append((raw_weight[i], i))
    
    biggest.sort(reverse=True)
    
    # Add top stocks until we reach the minimum
    for w, i in biggest:
        if i not in included_idx:
            included_idx.append(i)
        if len(included_idx) >= min_stocks:
            break

included_idx = sorted(included_idx)
selected_tickers = [tickers[i] for i in included_idx]

# Step 3: Final Optimization with Minimum Weights

Now we optimize the portfolio again using only our selected stocks, but with an additional constraint. Each included stock must have at least the weight derived from the weight fromula from before. In a real-world scenario, this ensures that positions are large enough to justify transaction costs, but here, we just do it because it's a requirement :)

In [ ]:
# DO IT AGAIN CVXPY 
# WE GETTING THE PROPER VALUES THIS TIMEEEE
new_num_stock = len(included_idx)
min_weight_if_included = 1 / (2 * new_num_stock)
weight_2 = cp.Variable(new_num_stock)

# Build sub-covariance matrix out of the stocks with actual weights
cov_sub = cov_matrix[np.ix_(included_idx, included_idx)]

second_objective = cp.Minimize(cp.quad_form(weight_2, cov_sub))

second_constraints = [
    cp.sum(weight_2) == 1,
    weight_2 >= min_weight_if_included,   
    weight_2 <= max_weight
]

# Add sector constraints again for the new portfolio (40% max per sector)
for sector in unique_sectors:
    indices = []
    for i in range(len(included_idx)):
        stock = tickers[included_idx[i]]
        if sectors[stock] == sector:
            indices.append(i)
    
    if indices:
        total_sector_weight = cp.sum(weight_2[indices])
        second_constraints.append(total_sector_weight <= max_sector_weight)

# Solve the new optimization problem
problem_2 = cp.Problem(second_objective, second_constraints)
final_results = problem_2.solve(solver=cp.SCS)

# Map the optimized weights back to the full ticker list
final_weights = np.zeros(num_tickers)
final_weights[included_idx] = np.maximum(np.array(weight_2.value).flatten(), 0)

# Normalize to ensure weights sum exactly to 1
final_weights = final_weights / final_weights.sum()
final_weights = np.minimum(final_weights, max_weight)

# Portfolio volatility
portfolio_vol = np.sqrt(final_weights @ cov_matrix @ final_weights)

# Make results df
results = pd.DataFrame({
    "Ticker": tickers,
    "Weight": final_weights
}).sort_values("Weight", ascending=False)

# Filters to remove all holdings with zero weights
results = results[results['Weight'] > 0].reset_index(drop=True)

print(f"Portfolio Volatility: {round(portfolio_vol,6)}")
print(f"Started with {len(tickers)} stocks, now have {len(results)} stocks.")
print(results)

In [ ]:
# If you're interested in seeing how the covariance matrix works or looks, here's a heatmap
# The darker the square, the less the two stocks move together
# Essentially the darker, the more the stocks balance each other's movements out

plt.figure(figsize=(8,6))
plt.imshow(cov_matrix, cmap="viridis")
plt.colorbar(label="Covariance")
plt.title("Covariance Matrix of All the Stocks")
plt.xticks(range(num_tickers), tickers, rotation=90, fontsize=6)
plt.yticks(range(num_tickers), tickers, fontsize=6)
plt.tight_layout()
plt.show()

# Final Portfolio Construction
 
Now we convert our optimal weights into actual shares we can buy (including fractional shares).


In [ ]:
# Step 0: Make adjusted weights dataframe
results['Weight'] = results['Weight'] / results['Weight'].sum()

# Step 1: Convert investment amount to USD
# This will make it easier when it comes to transaction costs and buying both canadian and american equities
#    At least that's what I thought when I first started, though it really became more of a nuisance
#        I'm still leaving it though since it works perfectly and I don't wanna mess anything up again ;-;
investment_cad = 1_000_000
usd_cad_ticker = yf.Ticker("CADUSD=X")
cad_to_usd = usd_cad_ticker.info.get('regularMarketPrice')  # Current exchange rate
investment_usd = investment_cad * cad_to_usd

print(f"Total Budget: ${investment_cad:,.2f} CAD = ${investment_usd:,.2f} USD")
print(f"Exchange Rate: 1 USD = {1/cad_to_usd:.4f} CAD")

# Step 2: Get current prices for each stock
current_prices = {}

for ticker in results['Ticker']:
    stock = yf.Ticker(ticker)
    price = stock.info.get('regularMarketPrice')
    current_prices[ticker] = price

# Step 3: Calculate allocation, shares, and fees
portfolio_data = []
total_fees_usd = 0

# First, calculate total fees
for ticker in results['Ticker']:
    weight = results[results['Ticker'] == ticker]['Weight'].values[0]
    price = current_prices[ticker]

    # Check if this is a Canadian stock
    is_canadian = ticker.endswith('.TO')

    # Amount allocated to this stock (in USD)
    allocated_usd = investment_usd * weight
    
    if is_canadian:
        # Convert allocated USD to CAD for Canadian stocks
        allocated_cad = allocated_usd / cad_to_usd
        shares_before_fee = allocated_cad / price

        # Calculate transaction fee (either $2.15 or $0.001 per share) in USD
        fee_cad = min(2.15, 0.001 * shares_before_fee)
        fee_usd = fee_cad * cad_to_usd
    else:
        # Is US equity

        # Calculate transaction fee (either $2.15 or $0.001 per share) in USD
        shares_before_fee = allocated_usd / price
        fee_usd = min(2.15, 0.001 * shares_before_fee)
    
    total_fees_usd += fee_usd

# Investable amount after fees
investable_usd = investment_usd - total_fees_usd

# Now allocate using the investable amount
for ticker in results['Ticker']:
    weight = results[results['Ticker'] == ticker]['Weight'].values[0]
    price = current_prices[ticker]

    # Check if this is a Canadian stock
    is_canadian = ticker.endswith('.TO')

    # Amount allocated to this stock (in USD)
    allocated_usd = investable_usd * weight
    
    if is_canadian:
        # Convert allocated USD to CAD for Canadian stocks
        allocated_cad = allocated_usd / cad_to_usd

        # Determine final amount of shares per stock
        shares_final = allocated_cad / price

        # How much it's worth in CAD
        value_cad = shares_final * price
        
        portfolio_data.append({
            'Ticker': ticker,
            'Price': price,
            'Currency': 'CAD',
            'Shares': shares_final,
            'Value (CAD)': value_cad,
            'Weight': weight  
        })
    else:
        # USA USA USA
        # Determine final amount of shares per stock
        shares_final = allocated_usd / price

        # How much it's worth in USD
        value_usd = shares_final * price

        # How much it's worth in CAD
        value_cad = value_usd / cad_to_usd  
        
        portfolio_data.append({
            'Ticker': ticker,
            'Price': price,
            'Currency': 'USD',
            'Shares': shares_final,
            'Value (CAD)': value_cad,  
            'Weight': weight  
        })

# Step 4: Create final DataFrame
Portfolio_Final = pd.DataFrame(portfolio_data)
Portfolio_Final.index = range(1, len(Portfolio_Final) + 1)

# Now all values are in CAD, so just sum them and stuff
total_invested_cad = Portfolio_Final['Value (CAD)'].sum()
total_fees_cad = total_fees_usd / cad_to_usd

# Display results
print(f"\nTotal Fees:        ${total_fees_cad:,.2f} CAD (${total_fees_usd:,.2f} USD)")
print(f"Total Invested:    ${total_invested_cad:,.2f} CAD")
print(f"Cash Remaining:    ${investment_cad - total_fees_cad - total_invested_cad:,.2f} CAD")
print(f"Weight Sum:        {Portfolio_Final['Weight'].sum():.6f}")
print(f"\n{Portfolio_Final.to_string()}\n")

# Step 5: Export CSV file
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']].copy()

group_number = "1259"  
Stocks_Final.to_csv(f"Stocks_Group_{group_number}.csv", index=False)

print(f"Saved Stocks_Group_{group_number}.csv")
print(f"\n{Stocks_Final.to_string(index=False)}")

# Before You Ask

Yes, we recognize if we wanted the **absolute lowest risk portfolio possible**, we could've just fed all of the tickers after being filtered into this optimizer. After all, in our tests, the optimization portion of the program only took 10 seconds to run even after being given 128 different stocks. However, we want a good grade. The competition results, frankly, matter the least to us. Being able to show this off on our resumes matters a ton, but nothing comes before grades. Hence, we've used metrics we discussed in class and explored in assignments, built a scoring algorithm (as one of our teammates have been able to build similar projects using them in the past), and applied ideas we learned in the presentations and past jupyter notebooks, to hopefully achieve our goal.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Wendi Xue, Ammar Adam, Maximilian Bartoszek